In [28]:
# Function is for validation generic code.
def validation(prompt = "",cast = None, cond = None, errMsg = None): # accepts four parameter for validating the user input
    # prompt is for string that will be visible to the user
    # cast is the casting of the input value the defalut value is string
    # cond is for condition it checks input with the condition passed by user, if no condition is passed then it chooses default value
    #errMsg is for custom error message  if no message is passed the it will choosed default value.
    while True: # This is infinity loop unless user enters correct value or unless user stops execution
        try: #this is try block for execution and check for error 
            res = (cast or str)(input(prompt)) # If cast is passed then this will check the user input satisfies or else will throw a error. User value are stored in res variable
            assert cond is None or cond(res) # Assert will check for the cond value if none then will return res or else check for the condition passed and if satisfied then return res else give error message
            return res # res has correct value and can be utilized for further computation
        except: # this is catch block if error is generated it will catch it and show the error message
            print(errMsg or "Invalid!!")   # if the error message is passed by user it will show that error message else will show Invalid +

In [29]:
# this function will accept file and read and convert in to the dataframe with Author and lines as the column name. the separator is the ':'
def fileToWork():
    import pandas as pd
    from os.path import exists as file_exists # check if the file exist
    path = validation("Enter the File Name",cast=str,cond= lambda x: x.endswith('txt')  and file_exists(x), errMsg= "file not proper") # '"poetry_lines.txt" is example user have to enter '
    data = pd.read_csv(path, sep=":",header = None, names=["Author", "lines"]) # convert into the dataframe with author and lines as the column name and return the data
    return data

In [30]:
# this function will create the dictionary
def createDict(lines):
    wordDict = {} # intialization
    word = lines.strip().split() # split the sentence and store to the list 
    for i in word: #iterate over the list of words collected from the list
        if i in wordDict: # convert list to dictionary and word as the key and occurrence count 
            wordDict[i] += 1
        else: # if occurr first time then add to the dictionary with 1 as occurrence    
            wordDict[i]  = 1
    return wordDict            # return the dictionary


In [31]:
# function will collect dictionary and the store the values of dictionary
def generateVector(data):
    # intialize the array to variable
    dictArray = [] 
    authors = []   
    vectorArray = [] 
    for i in range(len(data)): # iterate over the line given
        authors.append(data.iloc[i].Author.strip()) # append the author name to the list
        wordDict = createDict(data.iloc[i].lines) # function call for the getting the word dictionary
        vectorArray.append(list(wordDict.values())) # this will store the values for all the poems in the file
        dictArray.append(wordDict) # add dictionary to the array
    data["Dict"] = list(dictArray) # add the array to the data frame
    data["Vector"] = list(vectorArray)# add the array to the dataframe
# return the data frame
    return data


In [32]:
def userData(data):
    # import required libraries
    import numpy as np
    from numpy.linalg import norm
    simliar = 0 #intialize the counter
    userInput = input("Enter the sentence") #take input from the user
    userDict = createDict(userInput) #create dictionary of words and their occurrence for the user input line
    userVector = list(userDict.values()) # create the vector from the user entered dictionary
    for i in range(len(data)):
        ml = max(len(data.iloc[i].Vector),len(userVector)) # check for the max length from user line vector or file line vector
        A = np.concatenate((data.iloc[i].Vector , np.zeros(ml-len(data.iloc[i].Vector)))) # add zeros for the place that doesnt match
        B = np.concatenate((userVector , np.zeros(ml-len(userVector))))# add zeros
        # compute cosine similarity
        cosine = np.dot(A,B)/(norm(A)*norm(B)) # formula for the cosine similarity
        print(f"{data.iloc[i].Author}:{cosine:.2f}") # show cosine similarity for all the poets
        if simliar < cosine:
            simliar = cosine
            closest = i
    print("The closest poem is:\n", data.iloc[closest].Author,":",data.iloc[closest].lines)    



In [33]:
def main():
    data  = fileToWork()
    vectorData = generateVector(data)
    userData(vectorData)
main()

William Wordsworth:0.87
William Shakespeare:0.85
Robert Frost:0.95
The closest poem is:
 Robert Frost :  Whose woods these are I think I know./His house is in the village though;/He will not see me stopping here/To watch his woods fill up with snow.
